In [4]:
import pyterrier as pt
from pathlib import Path

if not pt.started():
    pt.init(tqdm="notebook")
    # pt.init()



dataset = pt.get_dataset('irds:trec-fair/2022/train')
indexer = pt.IterDictIndexer('D:/Q7/InformationRetrieval/group_project/test_dataset/indices_full/trec-fair_2022_multi', 
                             meta={"docno" : 20, "gender": 200, "title":512, "gender_category":100, "text":10000}, meta_reverse = ['docno'])



In [5]:
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['title', 'text', 'url'])

trec-fair/2022/train documents:   0%|          | 0/6475537 [00:00<?, ?it/s]

14:46:41.425 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 7232788 is not unique: 2621371,2596364
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:377)


In [6]:
index_ref = pt.IndexRef.of('D:/Q7/InformationRetrieval/group_project/test_dataset/indices_full/trec-fair_2022_multi')



In [10]:
import pandas as pd
import ast


def re_rank_by_gender(raw_df):
    df = raw_df.copy()
    df['gender'] = df['gender'].apply(lambda x: x.strip('[]').split(','))
    df['gender'] = df['gender'].apply(lambda x: x[0] if x else 'unknown')
    
    ranked_df = pd.DataFrame(columns=df.columns)
    
    desired_distribution = {'Man':0.3, 'Woman':0.3, 'Non-binary':0.3, 'Unknown':0.1, 'null':0.01}
    counts = {val: 0 for val in desired_distribution}
    
    while not df.empty:
        current_dist = {val: counts[val] / sum(counts.values()) if sum(counts.values()) > 0 else 0 for val in counts}
        next_att = max(desired_distribution.keys(), key=lambda x: desired_distribution[x] - current_dist.get(x, 0))
        
        if not df[df['gender_category'] == next_att].empty:
            next_doc = df[df['gender_category'] == next_att].sort_values(by='score', ascending=False).head(1)
            df = df.drop(next_doc.index)
            counts[next_att] += 1
            ranked_df = pd.concat([ranked_df, next_doc], ignore_index=True)
        else:
            # Remove the exhausted category and redistribute its target distribution
            del desired_distribution[next_att]
            if desired_distribution:  # Ensure there are still categories remaining
                total_distribution = sum(desired_distribution.values())
                for key in desired_distribution.keys():
                    desired_distribution[key] /= total_distribution
            else:
                break  # All categories are exhausted

    initial_score = 10
    score_decrement = 0.01
    ranked_df['score'] = initial_score - (ranked_df.index * score_decrement)
    ranked_df['rank'] = ranked_df.index
    
    return ranked_df

C:\Users\55241\AppData\Local\Temp\ipykernel_3636\1445925023.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ranked_df = pd.concat([ranked_df, next_doc], ignore_index=True)


,qid,docid,gender,title,gender_category,rank,score,query
0,1,716396,"""transgender male""",Shane Ortega,Man,0,10.00,transgender engineer
1,1,4571538,"""transgender female""",Grace Banu,Woman,1,9.99,transgender engineer
2,1,2631164,"""intersex""",Vijayarajamallika,Non-binary,2,9.98,transgender engineer
3,1,3286498,,LGBT rights in Tamil Nadu,Unknown,3,9.97,transgender engineer
4,1,1887532,"""transgender male""",Andrew Cray,Man,4,9.96,transgender engineer
...,...,...,...,...,...,...,...,...
995,1,6015553,,Hijron Ka Khanqah,Unknown,995,0.05,transgender engineer
996,1,419626,null,Namitha Marimuthu,null,996,0.04,transgender engineer
997,1,983625,null,Transgender rights,null,997,0.03,transgender engineer
998,1,4997878,null,Sasha Allen (The Voice 21),null,998,0.02,transgender engineer


In [105]:
pipeline = pt.BatchRetrieve(index_ref, wmodel='BM25', metadata=['gender', 'gender_category']) >> pt.apply.generic(re_rank_by_gender)
pipeline.search('scientist biography')

17:19:01.327 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


,qid,docid,gender,rank,score,query
0,1,5194694,"""male""",0,10.00,scientist biography
1,1,1001375,"""female""",1,9.99,scientist biography
2,1,5908501,"""male""",2,9.98,scientist biography
3,1,4638839,"""female""",3,9.97,scientist biography
4,1,1647209,"""male""",4,9.96,scientist biography
...,...,...,...,...,...,...
458,1,4065703,"""male""",458,5.42,scientist biography
459,1,5520877,"""female""",459,5.41,scientist biography
460,1,4255922,"""male""",460,5.40,scientist biography
461,1,1908308,"""female""",461,5.39,scientist biography
